In [2]:
! pip install pyspark torch torchvision

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing, load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from keras import layers, Input
from keras.utils import to_categorical

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.2/434.2 MB 31.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Using cached py4j-0.10.9.9-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached py4j-0.10.9.9-py2.py3-none-any.whl (203 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 MB 38.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 39.0 MB/s eta 0:00:00
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
  DEPRECATION: Building 'pyspark' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pysp

In [3]:
# Load and Normalize California Housing Data
housing = fetch_california_housing()
df_housing = pd.DataFrame(housing.data, columns=housing.feature_names)
df_housing['target'] = housing.target  # Apply log transformation

X = df_housing[housing.feature_names].values
y = df_housing['target'].values
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Convert NumPy arrays to PyTorch tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [7]:
# Define MLP architecture to match TensorFlow
class MLPRegressor(nn.Module):
    def __init__(self, input_size):
        super(MLPRegressor, self).__init__()
        # First layer with BatchNorm
        self.layer1 = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )
        # Second layer with ReLU and then Dropout (matching TensorFlow placement)
        self.layer2 = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.3)  # Moved dropout after ReLU to match TensorFlow
        )
        # Output layer
        self.layer3 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

# Create model
pt_model_regression = MLPRegressor(X_train.shape[1])

# Use Adam optimizer with default settings to match TensorFlow
# TensorFlow default Adam: lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-7
optimizer = optim.Adam(
    pt_model_regression.parameters(),
    lr=0.001,  # Default TensorFlow learning rate
    betas=(0.9, 0.999),  # Default betas
    eps=1e-8  # PyTorch default epsilon (slightly different from 

)

In [8]:
# Loss function
criterion = nn.MSELoss()

# Create DataLoader for mini-batch training
train_dataset = TensorDataset(X_train_t, y_train_t)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # Match TF batch size=32

# Training loop with mini-batches (50 epochs to match TensorFlow)
num_epochs = 50
for epoch in range(num_epochs):
    running_loss = 0.0

    # Mini-batch training
    for batch_x, batch_y in train_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = pt_model_regression(batch_x)
        loss = criterion(outputs, batch_y)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print statistics (similar to TensorFlow verbose=1)
    if epoch % 10 == 0 or epoch == num_epochs-1:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

Epoch [1/50], Loss: 0.9675
Epoch [11/50], Loss: 0.4346
Epoch [21/50], Loss: 0.3979
Epoch [31/50], Loss: 0.3803
Epoch [41/50], Loss: 0.3647
Epoch [50/50], Loss: 0.3596


In [9]:
# Evaluate model
pt_model_regression.eval()
with torch.no_grad():
    predictions = pt_model_regression(X_test_t)
    mse_loss = criterion(predictions, y_test_t)
    mae_loss = nn.L1Loss()(predictions, y_test_t)  # Calculate MAE to match TensorFlow metrics

print(f'Test Mean Squared Error: {mse_loss.item():.4f}')
print(f'Test Mean Absolute Error: {mae_loss.item():.4f}')

Test Mean Squared Error: 0.4928
Test Mean Absolute Error: 0.4592


In [10]:
# Make predictions with scaled input (matching TensorFlow's approach)
sample_input_np = np.array([[8.0, 41.0, 6.0, 1.0, 950.0, 4.0, 37.0, -122.0]])
sample_input_scaled = scaler.transform(sample_input_np)  # Apply scaling
sample_input_t = torch.tensor(sample_input_scaled, dtype=torch.float32)

pt_model_regression.eval()
with torch.no_grad():
    pt_prediction = pt_model_regression(sample_input_t).item()

print(f'Predicted House Price: {pt_prediction:.2f}')

Predicted House Price: 3.82


In [11]:
# Define MLP architecture
tf_model_regression = keras.Sequential([
    layers.Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Dense(32, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(1)
])

# Compile the model
tf_model_regression.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Display model summary
tf_model_regression.summary()

/opt/anaconda3/envs/ml-env/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,817 (11.00 KB)

 Non-trainable params: 128 (512.00 B)

In [12]:
# Train the model
tf_history_regression = tf_model_regression.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=32,
    verbose=1
)

Epoch 1/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 613us/step - loss: 1.0765 - mae: 0.7598 - val_loss: 0.5155 - val_mae: 0.5021
Epoch 2/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 410us/step - loss: 0.6246 - mae: 0.5776 - val_loss: 0.6097 - val_mae: 0.5396
Epoch 3/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 406us/step - loss: 0.5442 - mae: 0.5321 - val_loss: 0.5629 - val_mae: 0.5232
Epoch 4/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 433us/step - loss: 0.5055 - mae: 0.5107 - val_loss: 0.4253 - val_mae: 0.4455
Epoch 5/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 397us/step - loss: 0.4790 - mae: 0.4931 - val_loss: 0.4045 - val_mae: 0.4534
Epoch 6/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 397us/step - loss: 0.4512 - mae: 0.4800 - val_loss: 0.3795 - val_mae: 0.4233
Epoch 7/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 403us/step - loss: 0.4500 - mae: 0.4761 - val_loss: 0.3927 - val_mae: 0.4314
Epoch 8/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 472us/step - loss: 0.4336 - mae: 0.4670 - val_loss: 0.4150 - val_mae: 0.4442
Epoch 9/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 

In [13]:
# Evaluate the model on test data
loss, mae = tf_model_regression.evaluate(X_test, y_test)
print(f"Test Mean Absolute Error: {mae:.4f}")

129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 327us/step - loss: 0.3085 - mae: 0.4031
Test Mean Absolute Error: 0.4031
